In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

In [11]:
data_dir = 'skincancerimages/'
test_dir = os.path.join(data_dir, 'Test')
train_dir = os.path.join(data_dir, 'Train')

In [ ]:
classes = [ 'actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus',
            'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']

In [12]:
from glob import glob

In [ ]:
#Train data:
train_df = []

akeratosis_set = os.path.join(train_dir,'actinic keratosis')
basal_set = os.path.join(train_dir,'basal cell carcinoma')
dermaf_set = os.path.join(train_dir,'dermatofibroma')
melanoma_set = os.path.join(train_dir,'melanoma')
nevus_set = os.path.join(train_dir,'nevus')
pbkeratosis_set = os.path.join(train_dir,'pigmented benign keratosis')
skeratosis_set = os.path.join(train_dir,'seborrheic keratosis')
squamouscc_set = os.path.join(train_dir,'squamous cell carcinoma')
vlesions_set = os.path.join(train_dir,'vascular lesion')

#Image list:
normal_images = glob(normal_xray_set+"/*.jpeg")
pneumonia_images = glob(pneumonia_xray_set + "/*.jpeg")

#Normal X-Rays are labeled 0:
for img in normal_images:
    train_df.append((img, 0))

#Pneumonia X-Rays are labeled 1:
for img in pneumonia_images:
    train_df.append((img, 1))

#Creating a dataframe:
train_df = pd.DataFrame(train_df, columns=['X-Ray', 'Diagnosis'], index=None)

# testing models

In [8]:
image_size = 200
image_width, image_height = image_size, image_size
epochs = 20
batch_size = 32
test_size = 30

input_shape = (image_width, image_height, 3)

In [ ]:
inputs = input_shape

#1st Convolution
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

#2nd Convolution
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

#3rd Convolution
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

#4th Convolution
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

#5th Convolution
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Fully Connected layer
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(rate=0.3)(x)

# Output layer
output = Dense(units=1, activation='sigmoid')(x) #<--Converting a real value into a interpretable probability

# Creating model and compiling
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(filepath='best_weights.hdf5', save_best_only=True, save_weights_only=True)
#hdf5 (Hierarchical Data Format) is to store the weights of the best performing epoch
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max') #<--Reduce learning rate 
#when the metric has stopped improving.
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')